<div style="direction:rtl; text-align:right">

# المشروع الثاني: كم دراجة هوائية سيتم استئجارها خلال الشهر القادم ؟

<div style="direction:rtl; text-align:right">
أهلاً بكم في المشروع الثاني من علم البيانات. في هذا المشروع، سنكتشف سوية بيانات فريدة من نوعها وهي بيانات استئجار الدراجات الهوائية في مدينة واشنطن دي سي عاصمة الولايات المتحدة الأمريكية. يهدف هذا المشروع إلى التعمق في خطوات تحليل البيانات. الهدف من المشروع هو أولاً فهم البيانات ومن ثم محاولة نمذجة البيانات من خلال توقع كم دراجة هوائية سيتم استئجارها خلال الشهر القادم وذلك باستخدام بيانات السنين السابقة. البيانات موجودة بالكامل في موقع كاقل.

https://www.kaggle.com/c/bike-sharing-demand/data

<div style="direction:rtl; text-align:right">
في هذا الدرس، سوف نعمل الآتي: <br>
- قراءة البيانات <br>
- تنظيف وتنقيح البيانات <br>
- استكشاف البيانات <br>
- نمذجة البيانات لتوقع عدد مرات الإيجار

<div style="direction:rtl; text-align:right">
ملاحظة: عليك تعديل الخانات التي يوجد فيها الكود أدناه قبل تسليم المشروع. جميع الخانات الأخرى يجب ان تبقى كما هي بدون أي 
تعديل.

```
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<div style="direction:rtl; text-align:right">

# قراءة البيانات

In [ ]:
data = pd.read_csv('bike-train.csv')

In [ ]:
#لطباعة رأس الجدول نقوم باستخدام الأمر head
data.head()

<div style="direction:rtl; text-align:right">
في البداية، لنكتشف الصورة العامة لهذه البيانات. كم عدد الصفوف وكم عدد الأعمدة ؟ قم بكتابة الأوامر التي تؤدي هذه المهمة..

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

nrows = <YOUR CODE>
ncols = <YOUR CODE>

<div style="direction:rtl; text-align:right">
تمثل الصفوف قراءة البيانات لكل ساعة تقريباً خلال عامي 2011 و 2012. مع كل قراءة نجد العديد من البيانات الإضافية مثل: الموسم (season)، إجازة أم لا (holiday)، يوم عمل أم لا ؟ (workingday)، الطقس (weather)، درجة الحرارة (temp)، درجة الحرارة المحسوسة (atemp)، سرعة الرياح (windspeed)، عدد مرات الايجار من غير المسجلين (casual)، عدد مرات الايجار من المسجلين (registered)، مجموع مرات الإيجار (count).

<div style="direction:rtl; text-align:right">
سنقوم أولا بإلقاء نظرة على "أنواع" البيانات وما إن كان لدينا أي بيانات مفقودة أم لا..

In [ ]:
data.info()

<div style="direction:rtl; text-align:right">
هنا سنقوم بإخبار باندا بأن عمود datetime هو عبارة عن متغير من نوع تاريخ/وقت حتى نقوم بتنفيذ بعض العمليات لاحقاً

In [ ]:
data['datetime'] = pd.to_datetime(data['datetime'])

In [ ]:
data.info()

<div style="direction:rtl; text-align:right">
لنطبع مجموع مرات الإيجار خلال أول عشرة أيام:

In [ ]:
data[:24*10].plot(x='datetime', y='count') # 24 hours * 10 days

<div style="direction:rtl; text-align:right">

# تجهيز البيانات

<div style="direction:rtl; text-align:right">
الخطوة الأولى في تنظيف البيانات هي في تحويل البيانات الفئوية (مثل season, holiday, weather) إلى أعمدة مستقلة. نهتم كثيراً بهذه الخطوة خصوصاً في حالة النمذجة. سنستخدم الأمر get_dummies لأداء هذه المهمة.

In [ ]:
season_dummies = pd.get_dummies(data['season'], prefix='season', drop_first=False)
data = pd.concat([data, season_dummies], axis=1) # axis =1 means add new columns
weather_dummies = pd.get_dummies(data['weather'], prefix='weather', drop_first=False)
data = pd.concat([data, weather_dummies], axis=1) # axis =1 means add new columns

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

cols_to_drop = ['weather','season', 'casual', 'registered']
# drop columns here
#

<div style="direction:rtl; text-align:right">
الخطوة الثانية في تجهيز البيانات هي في تسوية البيانات (data scaling). وهذه الخطوة تهدف لتوحيد تباين البيانات. لاحظ أننا نقوم أولاً بحفظ المتوسط والانحراف المعياري، لكي نقوم باستخدامها لاحقاً لتسوية البيانات الجديدة.

In [ ]:
quant_features = ['temp', 'humidity', 'windspeed'] # those are all numerical columns..
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean)/std

<div style="direction:rtl; text-align:right">
حتى نتعرف على تأثير تسوية البيانات، سنقوم الآن بحساب المتوسط الحسابي والانحراف المعياري للأعمدة السابقة. ماذا تستنتج؟

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

for col in quant_features:
    mean = <YOUR CODE HERE>
    std  = <YOUR CODE HERE>
    print('mean of {} is {} and the standard deviation is {}'.format(col, mean, std))

In [ ]:
data.head()

<div style="direction:rtl; text-align:right">
أخيرا، الخطوة الثالثة هي في استخراج البيانات التاريخية مثل الساعة واليوم والشهر والسنة. كل هذا ممكن باستخدام أوامر مكتبة الباندا.

In [ ]:
data['hour'] = data['datetime'].dt.hour
data['day'] = data['datetime'].dt.day
data['month'] = data['datetime'].dt.month

<div style="direction:rtl; text-align:right">
ثم نقوم بحذف العمود الأصلي للوقت والتاريخ بعد استخراج المعلومات المفيدة منه.

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

col_to_delete = ['datetime']
# DELETE COLUMN NOW

In [ ]:
data.head()

<div style="direction:rtl; text-align:right">
سنقوم الآن بإعداد رسوم بيانية بسيطة لمتوسط عدد مرات الإيجار بالساعة وبالشهر. لإعداد هذه الرسوم، سنقوم أولاً بتلخيص البيانات باستخدام groupby ومن ثم رسم البيانات باستخدام plot:

In [ ]:
count_per_hour = data.groupby('hour')['count'].mean()

In [ ]:
count_per_hour.head()

In [ ]:
count_per_hour.plot(kind='bar')

#or..
#count_per_hour.plot.bar()

<div style="direction:rtl; text-align:right">
سنقوم الآن برسم مرات الإيجار حسب الشهر (month).

In [ ]:
############################
# عليك تعديل هذه الخانة قبل تسليم المشروع
# YOU HAVE TO EDIT THIS CELLL
############################

# <YOUR CODE HERE>

<div style="direction:rtl; text-align:right">

# نمذجة البيانات

<div style="direction:rtl; text-align:right">
سنقوم الآن بأول محاولة لنمذجة البيانات باستخدام ما يعرف بالانحدار الخطي (Linear Regression). ولكن قبل ذلك، سنقوم بتقسيم البيانات إلى بيانات للتدريب وبيانات للاختبار ومن ثم نمذجة البيانات واستخدام النموذج لتوقع مقدار الطلب على استئجار الدرجات الهوائية في بيانات جديدة ومن ثم تسليم التوقعات إلى موقع كاقل.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

<div style="direction:rtl; text-align:right">
أسهل وأسرع طريقة لمعرفة أداء النموذج هي باستخدام الأمر cross_val_score حيث يقوم هذا الأمر بتقسيم البيانات إلى بيانات تدريب وبيانات اختار باستخدام التحقق المتقاطع ومن ثم إجراء النمذجة بدون أي حاجة للتدخل. بعد ذلك، سنقوم بطباعة معدل الخطأ لنعرف بشكل عام عن متوسط أداء النموذج قبل تطبيقه على بيانات جديدة. سنتعرف بشكل مفصل على أساليب التحقق من النماذج لاحقاً:

In [ ]:
features = ['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed', 'season_1', 'season_2', 'season_3', 'season_4', 'weather_1', 'weather_2', 'weather_3', 'weather_4', 'hour', 'day', 'month']

In [ ]:
target = ['count']

In [ ]:
scores = cross_val_score(X=data[features],
                         y=data[target].values, 
                         estimator=DecisionTreeRegressor(),
                         scoring='neg_mean_squared_error',
                         cv=5)

In [ ]:
scores.mean()

In [ ]:
model = DecisionTreeRegressor()
model.fit(data[features], data[target])

<div style="direction:rtl; text-align:right">

# تسليم المشروع

<div style="direction:rtl; text-align:right">
تسليم هذا المشروع على مرحلتين: الأولى هي في تسليم الملف إلى موقع كاقل ومن ثم مشاركة الرابط، والمرحلة الثانية هي باستكمال الخانات التي تحتاج إلى إكمال ومن ثم تسليم المشروع إلى الموقع.

<div style="direction:rtl; text-align:right">
في كاقل، ستأتينا بيانات الاختبار، وهو جدول آخر فيه كل الأعمدة ما عدا عدد مرات الإيجار. ومهمتنا هي في استخدام بيانات التدريب لتوقع بيانات الاختبار، ومن ثم تسليم الحل إلى موقع كاقل لنرى كيف كان الأداء. لفعل ذلك، سنقوم بالتأكد من أننا نفذها جميع الخطوات السابقة في بيانات الاختبار كذلك.

In [ ]:
test = pd.read_csv('bike-test.csv')
test['datetime'] = pd.to_datetime(test['datetime'])
season_dummies = pd.get_dummies(test['season'], prefix='season', drop_first=False)
test = pd.concat([test, season_dummies], axis=1) # axis =1 means add new columns
weather_dummies = pd.get_dummies(test['weather'], prefix='weather', drop_first=False)
test = pd.concat([test, weather_dummies], axis=1) # axis =1 means add new columns
for each in quant_features:
    data.loc[:, each] = (data[each] - scaled_features[each][0]) / scaled_features[each][1]
test['hour'] = test['datetime'].dt.hour
test['day'] = test['datetime'].dt.day
test['month'] = test['datetime'].dt.month

<div style="direction:rtl; text-align:right">
وهنا سنقوم باستخدام النموذج السابق لتوقع مرات الإيجار في بيانات الاختبار:

In [ ]:
predictions = model.predict(test[features])

In [ ]:
test['count'] = predictions

In [ ]:
test[['datetime', 'count']].head()

<div style="direction:rtl; text-align:right">
الآن سنقوم بكتابة الملف حتى نقوم بتسليمها إلى كاقل. بعد كتابة الملف، اذهب إلى المسابقة في موقع كاقل، وسلم هذا الملف.

In [ ]:
test[['datetime', 'count']].to_csv('submission.csv', index=False)

<div style="direction:rtl; text-align:right">
في نهاية هذا المشروع، نتمنى أن تكون لدينا فكرة أوضح عما نفعله في علم البيانات: قراءة البيانات ثم تنظيف البيانات، ثم إجراء الاكتشاف الأولي للبيانات وبعد ذلك نقوم بنمذجة البيانات وتقييم النموذج. سنتعلم لاحقاً المزيد عن كل من هذه الخطوات.

<div style="direction:rtl; text-align:right">
إذا لم تكن راضيا عن أداء هذا النموذج، ندعوك إلى إلقاء نظرة على موقع كاقل والتعرف على الطرق المختلفة لتنظيف البيانات ورسم البيانات:

- https://www.kaggle.com/c/bike-sharing-demand/kernels